In [102]:
import pandas as pd
import numpy as np

In [103]:
df = pd.read_csv("dataset.csv")

In [104]:
columns_to_drop = list(df.loc[:, "CVA_tenderness":"chief_complaint"].columns) + \
                  list(df.loc[:, "arrival":"VITAMINS"].columns) + \
                  ['lang', 'employStatus', 'insurance_status', 'disposition', 'dispo', 
                   'UTI_diag', 'split', 'alt_diag', 'maritalStatus', 'race']

# Drop all the columns at once, except Urinary_tract_infections
df = df.drop(columns=[col for col in columns_to_drop if col != "Urinary_tract_infections"])

In [105]:
print(list(df.columns))

['ID', 'PATID', 'UCX_abnormal', 'ua_bacteria', 'ua_bili', 'ua_blood', 'ua_clarity', 'ua_color', 'ua_epi', 'ua_glucose', 'ua_ketones', 'ua_leuk', 'ua_nitrite', 'ua_ph', 'ua_protein', 'ua_rbc', 'ua_spec_grav', 'ua_urobili', 'ua_wbc', 'age', 'gender', 'ethnicity', 'Urinary_tract_infections', 'abxUTI']


In [106]:
df = df.drop(df[df['ua_clarity'] == 'not_reported'].index)
df = df.drop(df[df['ua_ph'] == 'not_reported'].index)
df['ua_urobili'] = df['ua_urobili'].replace('not_reported', 'negative')
df['ua_bacteria'] = df['ua_bacteria'].replace('none', 'negative')
df = df.replace('not_reported', 'negative')
df = df[df['age'] <= 60]
df.shape

(39491, 24)

In [107]:
df.head(20)

,ID,PATID,UCX_abnormal,ua_bacteria,ua_bili,ua_blood,ua_clarity,ua_color,ua_epi,ua_glucose,...,ua_protein,ua_rbc,ua_spec_grav,ua_urobili,ua_wbc,age,gender,ethnicity,Urinary_tract_infections,abxUTI
4,5,4,no,moderate,small,negative,clear,orange,negative,negative,...,moderate,small,1.030,negative,small,55,Male,Non-Hispanic,No,no
5,6,4,yes,few,small,large,not_clear,yellow,large,negative,...,small,moderate,1.030,negative,small,47,Female,Non-Hispanic,No,no
10,11,9,no,few,negative,negative,clear,yellow,small,negative,...,negative,negative,1.011,negative,moderate,31,Female,Hispanic or Latino,No,yes
11,12,10,yes,few,negative,small,not_clear,yellow,moderate,negative,...,negative,small,1.025,negative,moderate,54,Female,Non-Hispanic,No,no
14,15,12,no,negative,negative,negative,clear,yellow,negative,negative,...,negative,negative,1.012,negative,negative,24,Male,Non-Hispanic,No,yes
15,16,13,no,negative,negative,negative,clear,yellow,negative,negative,...,negative,negative,1.012,negative,negative,24,Male,Non-Hispanic,No,yes
16,17,13,yes,marked,negative,small,not_clear,yellow,large,negative,...,negative,small,1.028,negative,moderate,36,Female,Hispanic or Latino,No,yes
17,18,14,no,few,negative,moderate,not_clear,yellow,small,negative,...,negative,moderate,1.019,negative,small,46,Female,Non-Hispanic,No,no
19,20,16,no,many,small,large,not_clear,red,moderate,negative,...,small,large,1.027,negative,small,52,Female,Non-Hispanic,No,yes
21,22,18,yes,many,negative,negative,clear,orange,moderate,negative,...,negative,small,1.007,negative,small,35,Female,Non-Hispanic,No,yes


In [108]:
columns_to_check = ['ua_blood', 'ua_clarity', 'ua_color', 'ua_epi', 'ua_glucose', 
                    'ua_ketones', 'ua_leuk', 'ua_nitrite', 'ua_ph', 'ua_protein', 
                    'ua_rbc', 'ua_wbc', 'ua_bili']

# Filter the DataFrame to exclude rows where any of the columns has the value 'other'
clean_df = df[~df[columns_to_check].isin(['other']).any(axis=1)]


df.shape

(39491, 24)

In [109]:
yes_no_counts = clean_df['UCX_abnormal'].value_counts()

print(f"Yes: {yes_no_counts.get('yes', 0)}")
print(f"No: {yes_no_counts.get('no', 0)}")

Yes: 6421
No: 32492


In [110]:
yes_df = clean_df[clean_df['UCX_abnormal'] == 'yes']
no_df = clean_df[clean_df['UCX_abnormal'] == 'no']

no_df_sampled = no_df.sample(n=20000, random_state=42) 

balanced_df = pd.concat([yes_df, no_df_sampled])
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)


In [111]:
yes_no_counts = balanced_df['UCX_abnormal'].value_counts()

print(f"Yes: {yes_no_counts.get('yes', 0)}")
print(f"No: {yes_no_counts.get('no', 0)}")

Yes: 6421
No: 20000


In [112]:
total_count = len(balanced_df)
yes_count = 6421
no_count = 20000

# Define 30% of the total rows you want to sample
sample_size = int(0.30 * total_count)

# Calculate the number of 'Yes' and 'No' records to maintain the ratio
yes_ratio = yes_count / total_count
no_ratio = no_count / total_count

yes_sample_size = int(sample_size * yes_ratio)
no_sample_size = int(sample_size * no_ratio)

# Sample the respective number of 'Yes' and 'No' records
yes_sample = df[df['UCX_abnormal'] == 'yes'].sample(n=yes_sample_size, random_state=42)
no_sample = df[df['UCX_abnormal'] == 'no'].sample(n=no_sample_size, random_state=42)

# Combine the 'Yes' and 'No' samples into one DataFrame
test_data = pd.concat([yes_sample, no_sample])

clean_df_reset = balanced_df.reset_index(drop=True)
test_data_reset = test_data.reset_index(drop=True)


train_data = clean_df_reset.drop(index=test_data_reset.index)

# Export the sampled DataFrame to a CSV file
test_data.to_csv('test_data.csv', index=False)
train_data.to_csv('training_data.csv', index=False)

print(f"Sampled dataset with {yes_sample_size} 'Yes' and {no_sample_size} 'No' records has been created and exported.")

Sampled dataset with 1926 'Yes' and 5999 'No' records has been created and exported.


In [113]:
encoded_training = train_data.copy()
encoded_testing = test_data.copy()

In [114]:
# Binary Encoding for binary columns
binary_columns = {
    'UCX_abnormal': {'yes': 1, 'no': 0},
    'abxUTI': {'yes': 1, 'no': 0},
    'ua_urobili': {'positive': 1, 'negative': 0},
    'Urinary_tract_infections': {'Yes': 1, 'No': 0} 
}

for col, mapping in binary_columns.items():
    encoded_training[col] = encoded_training[col].map(mapping)
    encoded_testing[col] = encoded_testing[col].map(mapping)

# Ordinal Encoding for columns with natural order
ordinal_mappings = {
    'ua_bacteria': ['negative', 'few', 'moderate', 'marked', 'many'],
    'ua_bili': ['negative', 'small', 'moderate', 'large'],
    'ua_blood': ['negative', 'small', 'moderate', 'large'],
    'ua_clarity': ['clear', 'not_clear'],
    'ua_color': ['colorless', 'yellow', 'amber', 'orange', 'red'],
    'ua_epi': ['negative', 'small', 'moderate', 'large'],
    'ua_glucose': ['negative', 'small', 'moderate', 'large'],
    'ua_ketones': ['negative', 'small', 'moderate', 'large', '4+'],
    'ua_leuk': ['negative', 'small', 'moderate', 'large'],
    'ua_nitrite': ['negative', 'positive', 'other'],
    # 'ua_ph': ['5.0', '5.5', '6.0', '6.5', '7.0', '7.5', '8.0', '8.5', '9.0'],
    'ua_protein': ['negative', 'small', 'moderate', 'large'],
    'ua_rbc': ['negative', 'small', 'moderate', 'large'],
    'ua_wbc': ['negative', 'small', 'moderate', 'large']

}

for col, order in ordinal_mappings.items():
    encoded_training[col] = pd.Categorical(encoded_training[col], categories=order, ordered=True).codes
    encoded_testing[col] = pd.Categorical(encoded_testing[col], categories=order, ordered=True).codes

# One-Hot Encoding for categorical columns without a natural order
one_hot_cols = ['ethnicity']

# Use pd.get_dummies to one-hot encode the specified columns
encoded_training = pd.get_dummies(encoded_training, columns=one_hot_cols)
encoded_testing = pd.get_dummies(encoded_testing, columns=one_hot_cols)

encoded_training['Female'] = encoded_training['gender'].apply(lambda x: True if x == 'Female' else False)
encoded_training = encoded_training.drop(columns=['gender'])

encoded_testing['Female'] = encoded_testing['gender'].apply(lambda x: True if x == 'Female' else False)
encoded_testing = encoded_testing.drop(columns=['gender'])

encoded_training.head(50)

,ID,PATID,UCX_abnormal,ua_bacteria,ua_bili,ua_blood,ua_clarity,ua_color,ua_epi,ua_glucose,...,ua_wbc,age,Urinary_tract_infections,abxUTI,ethnicity_Hispanic or Latino,ethnicity_Non-Hispanic,ethnicity_Patient Refused,ethnicity_Unknown,ethnicity_negative,Female
7925,13977,9243,0,1,0,3,0,1,1,1,...,1,53,0,1,False,True,False,False,False,True
7926,884,576,0,0,0,0,0,1,0,0,...,0,50,0,0,False,True,False,False,False,False
7927,39389,26105,0,0,0,0,0,1,0,0,...,0,26,0,0,True,False,False,False,False,False
7928,46117,30879,0,1,0,2,1,4,0,0,...,1,43,0,0,True,False,False,False,False,False
7929,47879,32042,0,2,0,0,1,1,3,0,...,1,24,0,1,False,True,False,False,False,True
7930,35212,23265,0,0,0,0,0,1,0,0,...,0,55,0,0,False,False,False,False,True,False
7931,17511,11589,0,1,0,0,1,1,3,0,...,1,44,0,0,False,True,False,False,False,True
7932,12789,8425,0,2,0,0,0,3,1,0,...,2,23,1,1,False,True,False,False,False,True
7933,44991,30095,0,0,0,0,0,1,1,0,...,2,43,0,1,False,True,False,False,False,True
7934,73107,50391,0,0,0,0,0,1,0,0,...,0,54,0,0,False,True,False,False,False,False


In [115]:
encoded_testing.to_csv('encoded_testing.csv', index=False)
encoded_training.to_csv('encoded_training.csv', index=False)

In [116]:
encoded_testing.isnull().sum()

ID                              0
PATID                           0
UCX_abnormal                    0
ua_bacteria                     0
ua_bili                         0
ua_blood                        0
ua_clarity                      0
ua_color                        0
ua_epi                          0
ua_glucose                      0
ua_ketones                      0
ua_leuk                         0
ua_nitrite                      0
ua_ph                           0
ua_protein                      0
ua_rbc                          0
ua_spec_grav                    0
ua_urobili                      0
ua_wbc                          0
age                             0
Urinary_tract_infections        0
abxUTI                          0
ethnicity_Hispanic or Latino    0
ethnicity_Non-Hispanic          0
ethnicity_Patient Refused       0
ethnicity_Unknown               0
ethnicity_negative              0
Female                          0
dtype: int64